In [1]:
pwd

'/Users/arnav/Desktop/Y4/diss/diss/install/Jupiter'

In [16]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
# !pip install transformers
from transformers import BertTokenizer

In [3]:
df = pd.read_csv('test.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.head()

,seeker_post,response_post,level,rationales,joined
0,Help. Help me. I dunno what I'm doing anymore,"That's pretty vague, do you not know what you'...",0,NaN,Help. Help me. I dunno what I'm doing anymore ...
1,I'm done saying I love you to her because I do...,idk what a Red pill means exactly but my advic...,0,NaN,I'm done saying I love you to her because I do...
2,Always feel like I'm being criticized and mock...,"I think it's social anxiety , that creates par...",0,NaN,Always feel like I'm being criticized and mock...
3,My diet becomes fucked when i get depressed.. ...,By any chance do you think you're in a loop. J...,0,NaN,My diet becomes fucked when i get depressed.. ...
4,I hate not knowing why. I was diagnosed with d...,depression. not sadness which is caused by som...,0,NaN,I hate not knowing why. I was diagnosed with d...


In [4]:
# df_0 = df.loc[df['level']==0]
# df_1 = df[df['level']==1]
# df_2 = df[df['level']==2]

# df_0_downsampled = df_0.sample(df_1.shape[0])
# df_2_downsampled = df_2.sample(df_1.shape[0])

# df = pd.concat([df_0_downsampled, df_1, df_2_downsampled])

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [6]:
df['joined'].iloc[0]

"Help. Help me. I dunno what I'm doing anymore [SEP] That's pretty vague, do you not know what you're doing in regards to a specific section of your life? Like school or work?"

In [7]:
token = tokenizer.encode_plus(
    df['joined'].iloc[0], 
    max_length=256, 
    truncation=True, 
    padding='max_length', 
    add_special_tokens=True,
    return_tensors='tf'
)

Metal device set to: Apple M1 Pro


2023-02-06 02:00:00.373285: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-06 02:00:00.376906: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-06 02:00:00.377360: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
token.input_ids

<tf.Tensor: shape=(1, 256), dtype=int32, numpy=
array([[  101, 12056,   119, 12056,  1143,   119,   146,  3840,  9238,
         1186,  1184,   146,   112,   182,  1833,  4169,   102,  1337,
          112,   188,  2785, 14673,   117,  1202,  1128,  1136,  1221,
         1184,  1128,   112,  1231,  1833,  1107, 12747,  1106,   170,
         2747,  2237,  1104,  1240,  1297,   136,  2409,  1278,  1137,
         1250,   136,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 

In [9]:
X_input_ids = np.zeros((len(df), 256))
X_attn_masks = np.zeros((len(df), 256))

In [10]:
def generate_training_data(df_balanced, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df['joined'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [11]:
X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [12]:
labels = np.zeros((len(df), 3))
labels.shape

(9252, 3)

In [13]:
labels[np.arange(len(df)), df['level'].values] = 1 # one-hot encoded target tensor

In [14]:
labels

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [17]:
# creating a data pipeline using tensorflow dataset utility, creates batches of data for easy loading...
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
dataset.take(1) # one sample data

<TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(3,), dtype=tf.float64, name=None))>

In [18]:
def SentimentDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

In [19]:
dataset = dataset.map(SentimentDatasetMapFunction) # converting to required format for tensorflow dataset 

In [20]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(256,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.float64, name=None)}, TensorSpec(shape=(3,), dtype=tf.float64, name=None))>

In [21]:
dataset = dataset.shuffle(10000).batch(16, drop_remainder=True) # batch size, drop any left out tensor

In [22]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 3), dtype=tf.float64, name=None))>

In [23]:
p = 0.8
train_size = int((len(df)//16)*p) # for each 16 batch of data we will have len(df)//16 samples, take 80% of that for train.

In [24]:
train_size

462

In [25]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [26]:
from transformers import TFBertModel

In [27]:
model = TFBertModel.from_pretrained('bert-base-cased') # bert base model with pretrained weights

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [28]:
# defining 2 input layers for input_ids and attn_masks
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(3, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

sentiment_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
sentiment_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [29]:
optim = tf.keras.optimizers.legacy.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [30]:
sentiment_model.compile(optimizer=optim, loss=loss_func, metrics=[acc])

In [31]:
hist = sentiment_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=2
)

Epoch 1/2


2023-02-06 02:01:25.461915: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


462/462 [==============================] - ETA: 0s - loss: 0.8299 - accuracy: 0.6715

2023-02-06 02:11:21.063808: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


462/462 [==============================] - 642s 1s/step - loss: 0.8299 - accuracy: 0.6715 - val_loss: 0.8004 - val_accuracy: 0.6762
Epoch 2/2
462/462 [==============================] - 630s 1s/step - loss: 0.7872 - accuracy: 0.6761 - val_loss: 0.7481 - val_accuracy: 0.6783


In [32]:
sentiment_model.save('sentiment_model')

INFO:tensorflow:Assets written to: sentiment_model/assets


INFO:tensorflow:Assets written to: sentiment_model/assets


In [33]:
sentiment_model = tf.keras.models.load_model('sentiment_model')

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=['Non-Empathetic', 'Weakly Empathetic', 'Strongly Empathetic']):
    probs = model.predict(processed_data)[0]
#     return classes[np.argmax(probs)]
    return probs

In [35]:
input_text = input('Enter sentence here: ')
processed_data = prepare_data(input_text, tokenizer)
result = make_prediction(sentiment_model, processed_data=processed_data)
print(f"Predicted Sentiment: {result}")

Enter sentence here: Oh I feel really bad for you. I have been in the same situation as well.
1/1 [==============================] - 1s 722ms/step
Predicted Sentiment: [0.60198945 0.12849195 0.26951864]


In [ ]:
def predict_class(reviews):
    '''predict class of input text
      Args:
        - reviews (list of strings)
      Output:
        - class (list of int)
  '''
    return [np.argmax(pred) for pred in sentiment_model.predict(reviews)]


predict_class(reviews)

In [ ]:
# load blind set
test_set = pd.read_excel('/topics_rome_subset_test.xlsx')

test_set.head()

In [ ]:
from sklearn.metrics import classification_report


y_pred = predict_class(test_set["Text"])
print(classification_report(test_set["Labels"], y_pred))